In [1]:
import pandas as pd
import numpy as np

# Time series for a single day (24 hours)
hours = pd.date_range("2024-01-01", periods=24, freq='H')

# Energy Prices (simulated data)
energy_prices = pd.DataFrame({
    'Hour': hours,
    'Electricity_Price': np.random.uniform(0.05, 0.20, 24),  # $/kWh
    'Gas_Price': np.random.uniform(2.5, 4.0, 24)  # $/mmbtu
})

# Machine Energy Requirements
machines = pd.DataFrame({
    'Machine': ['Machine_1', 'Machine_2', 'Machine_3'],
    'Energy_Type': ['Electricity', 'Steam', 'Flexible'],  # Flexible means can use both
    'Hourly_Consumption': [100, 150, 120],  # kWh for electricity, lbs of steam
    'Efficiency_Electricity': [1.0, np.nan, 0.95],
    'Efficiency_Steam': [np.nan, 1.0, 0.90]
})

# Cogeneration Plant Capabilities
cogeneration = pd.DataFrame({
    'Plant': ['Cogeneration_1'],
    'Max_Electricity_Output': [500],  # kWh
    'Max_Steam_Output': [1000],  # lbs
    'Fuel_Consumption': [300],  # mmbtu
    'Operational_Cost': [0.03]  # $/kWh
})

# Operational Constraints
constraints = {
    'Production_Schedule': np.random.randint(1000, 1500, 24),  # Total energy requirement per hour
    'Transmission_Capacity_Electricity': 1000,  # Max kWh that can be transmitted per hour
    'Transmission_Capacity_Steam': 2000,  # Max lbs of steam that can be transmitted per hour
    'Prepurchased_Electricity': {hours[6]: 300, hours[18]: 400},  # kWh
}

# Display the first few rows of each dataset
print(energy_prices.head(), "\n")
print(machines, "\n")
print(cogeneration, "\n")
print("Constraints:", constraints)


                 Hour  Electricity_Price  Gas_Price
0 2024-01-01 00:00:00           0.138724   3.032654
1 2024-01-01 01:00:00           0.050462   2.573977
2 2024-01-01 02:00:00           0.056233   3.022879
3 2024-01-01 03:00:00           0.080829   3.008989
4 2024-01-01 04:00:00           0.128110   2.728594 

     Machine  Energy_Type  Hourly_Consumption  Efficiency_Electricity  \
0  Machine_1  Electricity                 100                    1.00   
1  Machine_2        Steam                 150                     NaN   
2  Machine_3     Flexible                 120                    0.95   

   Efficiency_Steam  
0               NaN  
1               1.0  
2               0.9   

            Plant  Max_Electricity_Output  Max_Steam_Output  Fuel_Consumption  \
0  Cogeneration_1                     500              1000               300   

   Operational_Cost  
0              0.03   

Constraints: {'Production_Schedule': array([1226, 1403, 1328, 1276, 1086, 1395, 1300, 1332, 13

In [2]:
import pulp

# Define the LP problem
problem = pulp.LpProblem("Energy_Cost_Optimization", pulp.LpMinimize)

# Variables
# Assuming the cogeneration plant can decide how much to produce within its capacity
electricity_generated = pulp.LpVariable("electricity_generated", lowBound=0, upBound=cogeneration['Max_Electricity_Output'][0])
steam_generated = pulp.LpVariable("steam_generated", lowBound=0, upBound=cogeneration['Max_Steam_Output'][0])
# Variables for electricity and steam purchased from external sources (for simplicity, assuming constant prices)
electricity_purchased = pulp.LpVariable("electricity_purchased", lowBound=0)
steam_purchased = pulp.LpVariable("steam_purchased", lowBound=0)

# Objective Function: Minimize costs
problem += (
    electricity_purchased * energy_prices['Electricity_Price'].mean() + 
    steam_purchased * energy_prices['Gas_Price'].mean() * 0.01 +  # Simplified conversion rate for steam cost
    electricity_generated * cogeneration['Operational_Cost'][0]
)

# Constraints
# Total energy requirement (simplified as a sum of electricity and steam requirements)
total_energy_requirement = machines['Hourly_Consumption'].sum()
problem += (electricity_generated + electricity_purchased + steam_generated + steam_purchased == total_energy_requirement)

# Transmission capacity constraints (simplified)
problem += (electricity_generated + electricity_purchased <= constraints['Transmission_Capacity_Electricity'])
problem += (steam_generated + steam_purchased <= constraints['Transmission_Capacity_Steam'])

# Pre-purchased electricity (simplified as averaged over the day)
avg_prepurchased_electricity = sum(constraints['Prepurchased_Electricity'].values()) / len(constraints['Prepurchased_Electricity'])
problem += (electricity_purchased >= avg_prepurchased_electricity)

# Solve the problem
problem.solve()

# Print the results
print("Status:", pulp.LpStatus[problem.status])
print("Electricity Generated (kWh):", pulp.value(electricity_generated))
print("Steam Generated (lbs):", pulp.value(steam_generated))
print("Electricity Purchased (kWh):", pulp.value(electricity_purchased))
print("Steam Purchased (lbs):", pulp.value(steam_purchased))
print("Total Cost ($):", pulp.value(problem.objective))


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/diogoribeiro/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_8/pmt00p0n2zsgw2rvt0djf05m0000gp/T/9ae39594e77949d8a361be30a5f1f0f4-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/_8/pmt00p0n2zsgw2rvt0djf05m0000gp/T/9ae39594e77949d8a361be30a5f1f0f4-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 22 RHS
At line 27 BOUNDS
At line 30 ENDATA
Problem MODEL has 4 rows, 4 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-4) rows, 0 (-4) columns and 0 (-9) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 43.283228
After Postsolve, objective 43.283228, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 43.28322787 - 0 iterations time 0.002, Presolve 0.00
Option for printingO